LANDMARK


In [2]:
%pip install pandas requests rapidfuzz


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import requests
from rapidfuzz import fuzz

API_KEY = "API_KEY"  # replace this
CSV_FILE = "sample1.csv"
OUTPUT_FILE = "verified_landmarks_output.csv"
RADIUS = 50
FUZZY_THRESHOLD = 85  


def get_nearby_places(lat, lon, radius, api_key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "key": api_key
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        place_names = [place["name"].lower() for place in data.get("results", [])]
        return place_names
    except Exception as e:
        print(f"API error: {e}")
        return []

def match_places_in_address(full_address, nearby_places, threshold):
    full_address = full_address.lower()
    matched = []
    for place in nearby_places:
        score = fuzz.partial_ratio(place, full_address)
        if score >= threshold:
            matched.append((place, score))
    return matched


def compute_confidence(matches):
    if not matches:
        return 0
    return min(100, 20 * len(matches))  # simple formula: 1 match = 20%, max 100%


def process_file(csv_file, api_key):
    df = pd.read_csv(csv_file)

    scores = []
    matches = []

    for i, row in df.iterrows():
        address = str(row['PrvAddress1'])
        lat = row['PrvLatitude']
        lon = row['PrvLongitude']

        nearby_places = get_nearby_places(lat, lon, RADIUS, api_key)
        matched_places = match_places_in_address(address, nearby_places, FUZZY_THRESHOLD)
        score = compute_confidence(matched_places)
        match_str = "; ".join([f"{p} ({s})" for p, s in matched_places])

        print(f"[{i+1}] Score: {score}% | Matches: {match_str}")
        scores.append(score)
        matches.append(match_str)

    df['Landmark_Match_Score'] = scores
    df['Matched_Landmarks'] = matches

    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\n✅ Done. Saved as {OUTPUT_FILE}")

process_file(CSV_FILE, API_KEY)


[1] Score: 0% | Matches: 
[2] Score: 0% | Matches: 
[3] Score: 0% | Matches: 
[4] Score: 20% | Matches: mumbai (100.0)
[5] Score: 20% | Matches: maitri park st (85.71428571428572)
[6] Score: 0% | Matches: 
[7] Score: 0% | Matches: 
[8] Score: 0% | Matches: 
[9] Score: 0% | Matches: 
[10] Score: 0% | Matches: 

✅ Done. Saved as verified_landmarks_output.csv


In [ ]:
import pandas as pd
import requests


API_KEY =  "API_KEY"  # replace this
CSV_FILE = "sample1.csv"
OUTPUT_FILE = "landmarks_appended.csv"
RADIUS = 300  # meters

def get_nearby_places(lat, lon, radius, api_key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "key": api_key
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        place_names = [place["name"] for place in data.get("results", [])]
        return place_names
    except Exception as e:
        print(f"API error: {e}")
        return []

def add_landmarks_to_file(csv_file, api_key):
    df = pd.read_csv(csv_file)

    landmark_columns = []

    for i, row in df.iterrows():
        lat = row['PrvLatitude']
        lon = row['PrvLongitude']

        landmarks = get_nearby_places(lat, lon, RADIUS, api_key)
        landmark_str = "; ".join(landmarks)

        landmark_columns.append(landmark_str)
        print(f"[{i+1}] Found {len(landmarks)} landmarks")

    df['Nearby_Landmarks'] = landmark_columns
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\n Done. Output saved to: {OUTPUT_FILE}")

add_landmarks_to_file(CSV_FILE, API_KEY)


[1] Found 20 landmarks
[2] Found 20 landmarks
[3] Found 20 landmarks
[4] Found 20 landmarks
[5] Found 20 landmarks
[6] Found 20 landmarks
[7] Found 20 landmarks
[8] Found 20 landmarks
[9] Found 20 landmarks
[10] Found 20 landmarks

✅ Done. Output saved to: landmarks_appended.csv


In [ ]:
import requests


API_KEY =  "API_KEY"  # replace this
RADIUS = 300  # meters


lat = 18.5204   # Pune center (example)
lon = 73.8567


def get_nearby_places(lat, lon, radius, api_key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if 'results' not in data:
        print("⚠️ No results found or error:", data)
        return []

    place_names = [place["name"] for place in data['results']]
    return place_names

# Run Test ===
landmarks = get_nearby_places(lat, lon, RADIUS, API_KEY)

print(f"\nNearby landmarks around ({lat}, {lon}):")
for i, name in enumerate(landmarks, 1):
    print(f"{i}. {name}")



Nearby landmarks around (18.5204, 73.8567):
1. Pune
2. Manish Clinic
3. Mukundrao Lele Hospital
4. Pune City Services
5. Walke Dr Dilip
6. Anuroop switch gear co.
7. Priyadarshaniya High School
8. Laxmi Sales and Services
9. Dr N P Rao Clinic
10. CSIPL
11. Priority Power Solutions
12. St. Anne’s Church
13. Fanuc India Pvt. Ltd
14. Quick Couriers and logistics pvt. ltd
15. Amritkar Vijay Eknath
16. Helping Hand Trust
17. Lala Urban Co-Operative Bank Limited Narayangao
18. Hi Skill Placement Services
19. Timpac Engineers
20. Kasba Peth
